### Connect to local cluster

In [1]:
from dask.distributed import Executor
e = Executor('localhost:8786')
e

<Executor: scheduler=localhost:8786 workers=1 threads=4>

### Functions for computation

These are simple toy functions that take a non-trivial amount of time.

In [2]:
from time import sleep

def slowinc(x):
    from random import random
    sleep(random())
    return x + 1

def slowdouble(x):
    from random import random
    sleep(random())
    return 2 * x
    
def slowadd(x, y):
    from random import random
    sleep(random())
    return x + y 

### Submit function to run on cluster

In [3]:
%time slowinc(10)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 1 s


11

In [4]:
future = e.submit(slowinc, 10)  # runs slowinc(10) on cluster
future

<Future: status: pending, key: slowinc-ad2dbadd22c92605400404687c25b93f>

In [5]:
future

<Future: status: finished, type: int, key: slowinc-ad2dbadd22c92605400404687c25b93f>

In [6]:
future.result()

11

### Submit many tasks

In [7]:
xs = [e.submit(slowinc, i) for i in range(16)]
ys = [e.submit(slowdouble, x) for x in xs]
zs = [e.submit(slowadd, x, y) for x, y in zip(xs, ys)]
zs

[<Future: status: pending, key: slowadd-3fc22ac2fded07b05486cbfee1e5ecd6>,
 <Future: status: pending, key: slowadd-87c458dfdfd4063148842840c42dfb7b>,
 <Future: status: pending, key: slowadd-08a01c67216caec44342f0508a8e3b20>,
 <Future: status: pending, key: slowadd-23a96221d2244a804e2da893412635df>,
 <Future: status: pending, key: slowadd-0fc30a053ba9e7ae2bec6e78f66417dc>,
 <Future: status: pending, key: slowadd-b90c8faa07899ef3f3a842f238ae7da3>,
 <Future: status: pending, key: slowadd-311fcf77b4fb1c8f1bcc70d3d0bbc541>,
 <Future: status: pending, key: slowadd-a4deb15159cd062091e807d39188cc49>,
 <Future: status: pending, key: slowadd-0c6c771986cfadb0508cda49412ec08d>,
 <Future: status: pending, key: slowadd-cf11b3538b16315fb1268c2168bc5a5f>,
 <Future: status: pending, key: slowadd-6441aa0bfd8db7c9057b97af3e5ef08c>,
 <Future: status: pending, key: slowadd-eea234a6ab042ed30cdd230c4675c5df>,
 <Future: status: pending, key: slowadd-bd0765c93402db71078c92c2f483f219>,
 <Future: status: pending

In [8]:
zs

[<Future: status: finished, type: int, key: slowadd-3fc22ac2fded07b05486cbfee1e5ecd6>,
 <Future: status: finished, type: int, key: slowadd-87c458dfdfd4063148842840c42dfb7b>,
 <Future: status: finished, type: int, key: slowadd-08a01c67216caec44342f0508a8e3b20>,
 <Future: status: finished, type: int, key: slowadd-23a96221d2244a804e2da893412635df>,
 <Future: status: finished, type: int, key: slowadd-0fc30a053ba9e7ae2bec6e78f66417dc>,
 <Future: status: finished, type: int, key: slowadd-b90c8faa07899ef3f3a842f238ae7da3>,
 <Future: status: finished, type: int, key: slowadd-311fcf77b4fb1c8f1bcc70d3d0bbc541>,
 <Future: status: finished, type: int, key: slowadd-a4deb15159cd062091e807d39188cc49>,
 <Future: status: finished, type: int, key: slowadd-0c6c771986cfadb0508cda49412ec08d>,
 <Future: status: finished, type: int, key: slowadd-cf11b3538b16315fb1268c2168bc5a5f>,
 <Future: status: finished, type: int, key: slowadd-6441aa0bfd8db7c9057b97af3e5ef08c>,
 <Future: status: finished, type: int, key:

In [9]:
e.gather(zs)

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48]

### Measure latency

In [10]:
%%time
future = e.submit(sum, zs)
print(future.result())

408
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 20.1 ms


### Custom computation

```
finish           total             single output
    ^          /        \
    |        z1          z2        neighbors merge
    |       /  \        /  \
    |     y1    y2    y3    y4     neighbors merge
    ^    / \   / \   / \   / \
start   x1 x2 x3 x4 x5 x6 x7 x8    many inputs
```

In [11]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = e.submit(slowadd, L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L                                       # swap old list for new

In [12]:
L

[<Future: status: pending, key: slowadd-89dce7c80b084c9abcddbb8a6b14ce6a>]

In [13]:
e.gather(L)

[408]